# Vectors and Matrices

There's no built in type for vectors and matrices in python.  So I'm going to use numpy

In [63]:
import numpy as np

vector = np.array([1, 2, 3, 4, 5, 6])
print(vector[0])
print(vector[1])

matrix = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

for el in matrix:
    print(el)


1
2
[1 2 3 4]
[5 6 7 8]
[ 9 10 11 12]


# Elimination

Going to try and write my own custom algorithm here, even though this and many other things are already built into numpy.

In [110]:
import numpy as np

def identity_matrix(size):
    matrix = np.zeros([size, size])
    pivot_row = 0
    while (pivot_row < size):
        pivot_col = pivot_row
        matrix[pivot_row][pivot_col] = 1

        pivot_row += 1
    
    return matrix

def row_swap(matrix, row_a, row_b):
    temp = matrix[row_a].copy()
    matrix[row_a] = matrix[row_b]
    matrix[row_b] = temp
    return matrix

# Takes a matrix and returns it in upper triangular form, along with number of row swaps required
def forward_elimination(matrix):    
    num_rows, num_cols = matrix.shape
    
    pivot_row = 0
    pivot_col = 0

    row_swaps = 0

    while (pivot_row < num_rows):
        pivot = matrix[pivot_row][pivot_col]
        if (pivot == 0):
            found_swap = False
            for swap_pivot_row in range(pivot_row + 1, num_rows):
                swap_pivot = matrix[swap_pivot_row][pivot_col]
                if (swap_pivot != 0):
                    found_swap = True
                    row_swaps += 1
                    row_swap(matrix, pivot_row, swap_pivot_row)
                    pivot = matrix[pivot_row][pivot_col]
                    break
            if not found_swap:
                raise Exception('Could not find pivot or row swap for {}').format(pivot_row)
        # We have a non zero pivot.  Now let's subtract off the rows below.
        for following_row in range(pivot_row + 1, num_rows):
            to_eliminate = matrix[following_row][pivot_col]
            if (to_eliminate != 0):
                # Only need to worry about nonzero elements below
                l = to_eliminate / pivot
                for elim_col in range(pivot_col, num_cols):
                    matrix[following_row][elim_col] = matrix[following_row][elim_col] - (l * matrix[pivot_row][elim_col])
        pivot_row = pivot_row + 1
        pivot_col = pivot_col + 1
    
    return matrix, row_swaps

# Brings us from upper triangular (row echelon) to reduced row echelon form.
def rref(matrix):
    num_rows, num_cols = matrix.shape

    pivot_row = num_rows - 1

    # Loop from bottom to top
    while (pivot_row >= 0):
        pivot_col = pivot_row
        pivot = matrix[pivot_row][pivot_col]
        if pivot == 0:
            raise Exception('Unexpected zero on row {}').format(pivot_row)
        
        # Divide out the row if pivot is not already 1
        if pivot != 1:
            for divide_col in range(pivot_col, num_cols):
                matrix[pivot_row][divide_col] = matrix[pivot_row][divide_col] / pivot
            pivot = matrix[pivot_row][pivot_col]
        
        # Subtract off the rows above
        for previous_row in reversed(range(0, pivot_row)):
            to_eliminate = matrix[previous_row][pivot_col]
            if (to_eliminate != 0):
                # Only need to worry about nonzero elements above
                l = to_eliminate / pivot 
                for elim_col in range(pivot_col, num_cols):
                    matrix[previous_row][elim_col] = matrix[previous_row][elim_col] - (l * matrix[pivot_row][elim_col])
        
        pivot_row = pivot_row - 1
    
    return matrix

# Takes a matrix and finds its inverse with Gaussian elimination
def matrix_inverse(matrix):
    num_rows, num_cols = matrix.shape
    matrix = np.append(matrix, identity_matrix(num_rows), axis=1)
    matrix, row_swaps = forward_elimination(matrix)
    matrix = rref(matrix)

    # Return just the inverse
    return matrix[0:num_rows, num_cols:]

matrix = np.array([[1, 2, 3], [5, 6, 7], [9, 10, 12]])
print(matrix_inverse(matrix))


[[-0.5  -1.5   1.  ]
 [-0.75  3.75 -2.  ]
 [ 1.   -2.    1.  ]]


# Determinants

## Pivot Algorithm

Perform forward elimination to get $A$ to $U$.  If an odd number of row exchanges are involved, then multiply answer by $-1$. The answer being the product of the diagonal pivots.


In [111]:
def diagonal_product(matrix):
    num_rows = matrix.shape[0]

    product = 1

    pivot_row = 0
    pivot_col = 0

    while (pivot_row < num_rows):
        pivot = matrix[pivot_row][pivot_col]
        product = product * pivot

        pivot_row += 1
        pivot_col +=1
    
    return product

def determinant_by_pivots(matrix):
    num_rows, num_cols = matrix.shape
    if (num_rows != num_cols):
        raise Exception('Determinants are only for square matrices')

    matrix_u, row_swaps = forward_elimination(matrix)
    product = diagonal_product(matrix_u)

    if (row_swaps % 2 == 1):
        product = product * -1
    
    return product

matrix = np.array([[0, 0, 1], [0, 2, 3], [4, 5, 6]])
print(determinant_by_pivots(matrix))

-8


## The Big Formula for Determinants

For this one we split the determinant into selections from each column and row.  We can then extract the multipliers, and pull the sign from how many row swaps are required (even or odd) for the remaining permutation matrix.  We then add all those determinants together to get our determinant.


In [112]:
from itertools import permutations

# Takes a permutation matrix and returns its determinant based on number of swaps to reach identity matrix
def perm_determinant(permutation_matrix):
    num_rows, num_cols = permutation_matrix.shape

    pivot_row = 0
    pivot_col = 0

    row_swaps = 0

    while (pivot_row < num_rows):
        pivot = permutation_matrix[pivot_row][pivot_col]
        if pivot != 0 and pivot != 1:
            raise Exception("Only expecting 0s and 1s in a permutation matrix")

        if (pivot == 0):
            found_swap = False
            for swap_pivot_row in range(pivot_row + 1, num_rows):
                swap_pivot = permutation_matrix[swap_pivot_row][pivot_col]
                if swap_pivot != 0 and swap_pivot != 1:
                    raise Exception("Only expecting 0s and 1s in a permutation matrix")
                if (swap_pivot != 0):
                    found_swap = True
                    row_swaps += 1
                    row_swap(permutation_matrix, pivot_row, swap_pivot_row)
                    break
            if not found_swap:
                raise Exception('Could not find pivot or row swap for {}').format(pivot_row)
        
        pivot_row = pivot_row + 1
        pivot_col = pivot_col + 1
    
    if (row_swaps % 2 == 1):
        return -1
    else:
        return 1

def determinant_by_big_formula(matrix):
    num_rows, num_cols = permutation_matrix.shape
    perms = permutations(range(0,num_cols))

    determinant = 0

    for perm in perms:
        product = 1
        perm_matrix = np.zeros(permutation_matrix.shape)
        row = 0
        for col in perm:
            el = matrix[row][col]
            product = product * el
            perm_matrix[row][col] = 1
            row += 1
        product = product * perm_determinant(perm_matrix)
        determinant += product

    return determinant

print(determinant_by_big_formula(np.array([[0, 0, 1], [0, 2, 3], [4, 5, 6]])))

-8
